# Imports

In [2]:
import torch
import pandas as pd
from src.stratSampling_byAge_KFold import *
from src._cnn_config import *

# Configurations

In [3]:
random_seed = 77777
filenames = torch.load('/home/yhchoi/24BrainMRI_Data/Labels/Internal/filename_labels_internal.pt')
age = torch.load('/home/yhchoi/24BrainMRI_Data/Labels/Internal/age_labels_internal_nodupes.pt')
pathogens = torch.load('/home/yhchoi/24BrainMRI_Data/Labels/Internal/pathogen_labels_internal_nodupes.pt')

id_lookup_dir = '/home/yhchoi/FSTrain_0205/src/labels/id_lookups.csv'
prognosis_labels_dir = '/home/yhchoi/FSTrain_0205/src/labels/prognosis_labels_tidiedup.csv'

# Fold Construction (Internal)

In [4]:
df = pd.read_csv(id_lookup_dir)
file_id_dict = df.set_index('Filename')['ID'].to_dict()
df2 = pd.read_csv(prognosis_labels_dir)
age_id_dict = df2.set_index('ID')['age'].to_dict()
pathogen_id_dict = df2.set_index('ID')['label'].to_dict()

def pathogen_indices_extraction(age, pathogens, pathogen_to_extract):
    pathogen_groups = torch.zeros_like(pathogens)

    pathogen_groups[pathogens == pathogen_to_extract] = 1
    pathogen_groups[pathogens != pathogen_to_extract] = 0

    indices_toextract = torch.where(pathogen_groups == 1)[0]

    age_extracted = age[indices_toextract]

    return indices_toextract, age_extracted

for pathogen in [0, 1, 2, 3]:
    ind_e, age_e = pathogen_indices_extraction(age, pathogens, pathogen)
    a = Strat_5fold_Split_Indices_ByAge(ind_e, age_e, random_state=random_seed)
    torch.save(a, 'kfoldsplitindices_ID_' + str(random_seed) + '_' + str(pathogen) + '.pt')

def filename_indices_from_IDs(filenames, file_id_list):
    indices = [filenames.index(file_id) for file_id in file_id_list]
    return indices

df = pd.read_csv('/home/yhchoi/24BrainMRI_Data/Labels/filename_id_internal.csv')

file_id_no_dupes = torch.load('/home/yhchoi/24BrainMRI_Data/Labels/Internal/id_labels_internal_nodupes.pt')

for pathogen in [0, 1, 2, 3]:
    filenamelist_list = []
    b = torch.load('kfoldsplitindices_ID_77777_' + str(pathogen) + '.pt')
    c = [[file_id_no_dupes[i] for i in b[j]] for j in range(5)]
    for IDlist in c:
        filenamelist = []
        for ID in IDlist:
            selected_values = df['Filename'][df['ID'] == int(ID)].to_list()
            filenamelist += selected_values
        filenamelist_list.append(filenamelist)
    torch.save(filenamelist_list, 'kfoldsplitfilenames_' + str(random_seed) + '_' + str(pathogen) + '.pt')
    filenameidxlist_list = [torch.tensor(filename_indices_from_IDs(filenames, filenamelist_list[i])) for i in range(5)]
    print(filenameidxlist_list)
    torch.save(filenameidxlist_list, 'kfoldsplitindices_' + str(random_seed) + '_' + str(pathogen) + '.pt')

# Integrity Check

In [2]:
y = torch.load('/home/yhchoi/24BrainMRI_Data/Labels/Internal/prognosis_labels_internal.pt')
print(len(y))
for pathogen in [0, 1, 2, 3]:
    b = torch.load('kfoldsplitindices_' + str(random_seed) + '_' + str(pathogen) + '.pt')
    y1 = [y[b[i]] for i in range(5)]
    for k in y1:
        print(k)
    print('####################################')

413
tensor([4, 4, 2, 2, 3, 4, 3, 3, 6, 1, 2, 2, 2, 2, 2, 2, 2])
tensor([1, 1, 1, 4, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 4, 1])
tensor([1, 4, 4, 4, 4, 4, 1, 2, 2, 2, 0, 5, 0])
tensor([1, 1, 2, 2, 2, 2, 2, 4, 4, 4, 0, 2, 2])
tensor([1, 1, 1, 5, 5, 5, 5, 0, 1, 0, 0, 4, 4, 4])
####################################
tensor([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 4, 4, 1, 0, 4, 1])
tensor([0, 0, 1, 1, 0, 1, 1, 1, 5, 1, 2])
tensor([1, 1, 6, 0, 4, 5, 1, 1, 1, 1, 5, 1, 1, 1])
tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 5])
tensor([1, 5, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1])
####################################
tensor([1, 1, 1, 5, 5, 1, 1, 1, 3, 3, 1, 1, 1])
tensor([5, 0, 1, 2, 1, 1, 1, 4, 4])
tensor([1, 0, 3, 1, 0, 2, 2])
tensor([4, 1, 2, 1, 0])
tensor([1, 1, 1, 0, 6, 6, 6, 5, 4])
####################################
tensor([1, 1, 2, 0, 0, 0, 1, 0, 1, 4, 0, 0, 1, 2, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 0, 1, 0, 1, 3, 5, 1, 1, 1])
tensor([0, 1, 1, 1, 0, 0, 1, 1,

In [15]:
for pathogen in [0, 1, 2, 3]:
    a = torch.load('kfoldsplitindices_' + str(random_seed) + '_' + str(pathogen) + '.pt')
    print(sum([len(a[i]) for i in range(5)]))

73
75
43
222
